In [1]:
# Necessary imports
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
df_pos = pd.read_csv(open('clean_positive_train.csv','r'), encoding='utf-8', engine='c')
df_neg = pd.read_csv(open('clean_negative_train.csv','r'), encoding='utf-8', engine='c')

df_pos['text'] = df_pos['text'].astype(str)
df_pos['parent_text'] = df_pos['parent_text'].astype(str)

df_neg['text'] = df_neg['text'].astype(str)
df_neg['parent_text'] = df_neg['parent_text'].astype(str)

In [3]:
df_pos.describe()

,score,ups,controversiality,parent_score,parent_ups,parent_controversiality
count,99999.0,99999.000000,99999.000000,99999.000000,99999.000000,99999.00000
mean,1.0,198.155082,0.000020,0.990770,368.973070,0.00064
std,0.0,256.334734,0.004472,0.095629,535.679712,0.02529
min,1.0,66.000000,0.000000,0.000000,-8907.000000,0.00000
25%,1.0,83.000000,0.000000,1.000000,84.000000,0.00000
50%,1.0,116.000000,0.000000,1.000000,184.000000,0.00000
75%,1.0,200.000000,0.000000,1.000000,417.000000,0.00000
max,1.0,5488.000000,1.000000,1.000000,9531.000000,1.00000


In [4]:
df_neg.describe()

,score,ups,controversiality,parent_score,parent_ups,parent_controversiality
count,99998.0,99998.000000,99998.000000,99998.000000,99998.000000,99998.000000
mean,0.0,-14.583632,0.001490,0.912518,67.440239,0.002720
std,0.0,15.649932,0.038572,0.282541,219.047635,0.052083
min,0.0,-1077.000000,0.000000,0.000000,-1622.000000,0.000000
25%,0.0,-15.000000,0.000000,1.000000,6.000000,0.000000
50%,0.0,-10.000000,0.000000,1.000000,15.000000,0.000000
75%,0.0,-8.000000,0.000000,1.000000,44.000000,0.000000
max,0.0,-6.000000,1.000000,1.000000,14776.000000,1.000000


We now have a a fraction of our dataset cleaned and loaded. The next step is to combine the two datasets, and shuffle them. After that we divide the datasets in a train and test set.

In [5]:
# First we concatenate both lists and shuffle it to scrabble positive and negative
df = pd.concat([df_pos, df_neg])
df = df.sample(frac=1).reset_index(drop=True)
df.dropna(axis=0, inplace=True)

# Second we split our data for training and testing
df['combined'] = df[['text', 'parent_text']].apply(lambda x: ' '.join(x), axis=1)

text_data = df['combined']
text_score = df['score']
# parent_text_data = df['parent_text']
# parent_text_score = df['parent_score']
X_train, X_test, y_train, y_test = train_test_split(text_data,text_score, test_size = 0.20, random_state = 42)

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

X_train: (159997,)
X_test: (40000,)
y_train: (159997,)
y_test: (40000,)


Now we use a tokenizer which fits all the text in our dataset, it then assigns an integer to each learned word which allows us to convert each entry to a sequence of numbers. These sequences can then be easily passed to our neural network

In [6]:
# We create a tokenizer which will give a word_index integer value to each word
tokenizer = Tokenizer(num_words=10000, lower=True, split=' ', document_count=0)

# Create the word_index list based on all our data
tokenizer.fit_on_texts(text_data)

# Now we make a list of sequences of integers based on our texts
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

Because each sentence has a different length and we want to pass the same length vector to our neural network every time, we pad them adding zeros at the end of each sequence so each is 128 integers long.

In [7]:
train_data = keras.preprocessing.sequence.pad_sequences(X_train_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=128)

test_data = keras.preprocessing.sequence.pad_sequences(X_test_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=128)

In [8]:
print(len(train_data[0]), len(train_data[1]))

128 128


## Neural networks!

Now we can start to build our neural network with TensorFlow. First we create an embedding layer which turns positive integers (indexes) into dense vectors of fixed size. 
After that we use a GlobalAveragePooling layer which averages all input, this is needed because we feed vectors that contain a lot of zeros, otherwise a lot of neurons will never fire. 
Then have 2 deep learning layers and in the end one node which will return whether the text that is passed will get a positive or negative score. 

In [9]:
vocab_size = len(tokenizer.word_index) + 1

model = tf.keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, 4))
model.add(keras.layers.GlobalAveragePooling1D())
#model.add(keras.layers.Dense(4, activation=tf.nn.relu))
#model.add(keras.layers.Dense(8, activation=tf.nn.relu))
model.add(keras.layers.Dense(2, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 4)           868984    
_________________________________________________________________
global_average_pooling1d (Gl (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 2)                 10        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3         
Total params: 868,997
Trainable params: 868,997
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [11]:
history = model.fit(train_data,
                    y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(test_data, y_test),
                    verbose=1)

Train on 159997 samples, validate on 40000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/40
159997/159997 [==============================] - 3s 17us/sample - loss: 0.6888 - acc: 0.5485 - val_loss: 0.6826 - val_acc: 0.5990
Epoch 2/40
159997/159997 [==============================] - 2s 15us/sample - loss: 0.6695 - acc: 0.6194 - val_loss: 0.6573 - val_acc: 0.6579
Epoch 3/40
159997/159997 [==============================] - 2s 16us/sample - loss: 0.6414 - acc: 0.6637 - val_loss: 0.6348 - val_acc: 0.6769
Epoch 4/40
159997/159997 [==============================] - 3s 16us/sample - loss: 0.6198 - acc: 0.6812 - val_loss: 0.6198 - val_acc: 0.6767
Epoch 5/40
159997/159997 [==============================] - 2s 15us/sample - loss: 0.6049 - acc: 0.6912 - val_loss: 0.6106 - val_acc: 0.6841
Epoch 6/40
159997/159997 [==============================] - 2s 15us/sample - loss: 0.5939 - acc: 0.6987 - val_loss: 0.6052 - val_acc: 0.6868
Epoch 7/40
159997/159997 [=============================

As we can see our neural network achieves an accuracy of about 73% on our training data and 68% on our test data. Also notice that the accuray of the validation set goes down while that of the training goes up, meaning that we're overfitting our network.